In [1]:
import duckdb
import pandas as pd

In [3]:
sql_query = '''
show tables
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name


In [4]:
sql_query_import_1 = '''
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT *
FROM read_csv_auto(
  'data/dof_parking_violation_codes.csv',
  normalize_names=True
  )
'''

sql_query_import_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT *
FROM read_csv_auto(
  'data/parking_violations_issued_fiscal_year_2023_sample.csv',
  normalize_names=True
  )
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
  con.sql(sql_query_import_1)
  con.sql(sql_query_import_2)

In [13]:
sql_query = '''
  SELECT *
  FROM parking_violation_codes
  WHERE manhattan_96th_st_below != all_other_areas
  LIMIT 5
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,code,definition,manhattan_96th_st_below,all_other_areas
0,20,NO PARKING-DAY/TIME LIMITS,65,60
1,23,NO PARKING-TAXI STAND,65,60
2,24,NO PARKING-EXC. AUTH. VEHICLE,65,60
3,32,OT PARKING-MISSING/BROKEN METR,65,35
4,34,EXPIRED METER,65,35


In [15]:
sql_query = '''
show tables
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name
0,first_model
1,parking_violation_codes
2,parking_violations_2023
3,ref_model


In [16]:
sql_query = '''
SELECT * FROM ref_model
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,count_star()
0,97


In [4]:
sql_query = '''
-- show tables;
SELECT * FROM bronze_parking_violations LIMIT 5;
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())


,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,vehicle_expiration_date,violation_location,violation_precinct,issuer_precinct,issuer_code,issuer_command,issuer_squad,violation_time,violation_county,violation_legal_code,vehicle_color,vehicle_year
0,9010912681,CA,PAS,2022-10-11,17,SUBN,FORD,T,20220788,50.0,50,50,365910,T201,D,0825A,BX,NaN,BLACK,0
1,4858762841,NY,PAS,2023-08-21,36,4DSD,HONDA,V,0,NaN,0,0,0,None,None,1036A,BK,True,GY,2003
2,4854645684,FL,PAS,2023-07-26,36,UT,BMW,V,0,NaN,0,0,0,None,None,0126P,QN,True,WHI,2022
3,9044582707,NY,PAS,2023-04-10,21,SUBN,SUBAR,T,20231217,79.0,79,79,370326,T301,H,0924A,K,NaN,GY,2017
4,9041503330,NY,PAS,2023-03-21,21,4DSD,CHEVR,T,20250320,26.0,26,26,373097,T103,C,1155A,NY,NaN,BK,2018


In [5]:
sql_query = '''
-- show tables;
-- SELECT * FROM silver_parking_violation_codes LIMIT 5;
SELECT * FROM silver_violation_tickets LIMIT 5;
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,summons_number,issue_date,violation_code,is_manhattan_96th_st_below,issuing_agency,violation_location,violation_precinct,issuer_precinct,issuer_code,issuer_command,issuer_squad,violation_time,violation_county,violation_legal_code,fee_usd
0,9010912681,2022-10-11,17,False,T,50.0,50,50,365910,T201,D,0825A,BX,NaN,95
1,4858762841,2023-08-21,36,False,V,NaN,0,0,0,None,None,1036A,BK,True,50
2,4854645684,2023-07-26,36,False,V,NaN,0,0,0,None,None,0126P,QN,True,50
3,9044582707,2023-04-10,21,False,T,79.0,79,79,370326,T301,H,0924A,K,NaN,65
4,9041503330,2023-03-21,21,False,T,26.0,26,26,373097,T103,C,1155A,NY,NaN,65


In [10]:
sql_query = '''
-- show tables;
-- SELECT * FROM gold_ticket_metrics LIMIT 5;
SELECT * FROM gold_vehicles_metrics LIMIT 5;
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,registration_state,ticket_count
0,NJ,9258
1,PA,3514
2,FL,2414
3,CT,1787
4,GA,840


In [11]:
sql_query = '''
SELECT
    violation_code,
    SUM(fee_usd) AS total_revenue_usd
FROM
    silver_parking_violation_codes
GROUP BY
    violation_code
HAVING
    NOT(total_revenue_usd >= 1)
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,violation_code,total_revenue_usd
0,41,0.0


In [13]:
sql_query = '''
select * from "nyc_parking_violations"."main_dbt_test__audit"."violation_codes_revenue"
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,violation_code,total_revenue_usd
0,41,0.0


In [16]:
sql_query_import_1 = '''
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT *
FROM read_csv_auto(
  'data/dof_parking_violation_codes.csv',
  normalize_names=True
  )
'''

sql_query_import_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT *
FROM read_csv_auto(
  'data/parking_violations_issued_fiscal_year_2023_sample.csv',
  normalize_names=True
  )
'''

with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
  con.sql(sql_query_import_1)
  con.sql(sql_query_import_2)

sql_query = '''
show tables
'''

with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name
0,parking_violation_codes
1,parking_violations_2023


In [18]:
sql_query = '''
show tables
'''

with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,first_model
3,gold_ticket_metrics
4,gold_vehicles_metrics
5,parking_violation_codes
6,parking_violations_2023
7,ref_model
8,silver_violation_tickets
9,silver_violation_vehicles
